In [10]:
%matplotlib inline
import spikeinterface.full as si
import spikeinterface.extractors as se
# import numpy as np
# import matplotlib.pyplot as plt
from pathlib import Path


In [151]:
def construct_sg_file(base_folder):
    base_folder = Path(base_folder)
    # Correctly append only the probe file to the directory without repeating the base folder name
    folder_path = base_folder / f'kilosort4'
    return folder_path

#kilosort4の出力があるフォルダを指定する
# base_folder = r'O:\seaturtle\2023\combine\alpha\20230819\20230819_164611.kilosort\stFolder'
# base_folder = r'O:\seaturtle\2023\combine\beta\20230820\20230820_151453.kilosort\stFolder'
base_folder = r'O:\seaturtle\2023\combine\gamma\20230821-2\20230821_164906.kilosort\stFolder'

folder_path = construct_sg_file(base_folder)
print(folder_path)
print(base_folder)

O:\seaturtle\2023\combine\gamma\20230821-2\20230821_164906.kilosort\stFolder\kilosort4
O:\seaturtle\2023\combine\gamma\20230821-2\20230821_164906.kilosort\stFolder


In [152]:
# phy_file
#kilosort4の結果を読み取る
# sorting=se.read_phy(folder_path)

#noise,goodなどをphyで設定済みならば、こちらをつかう
# sorting=se.read_phy(folder_path, exclude_cluster_groups=['noise'])
sorting = se.read_phy(folder_path  / 'mPhy', exclude_cluster_groups=['noise'])

In [153]:
#base_folderには、traces_cached_seg0.rawファイルがあると仮定する
#spikeinterfaceでshift補正したデータ読み取り
rec2 = si.load_extractor(base_folder)

In [154]:
#ecephys_piplelineのpost_processingの代わり
rec2.annotate(is_filtered=True)
# sorting2=si.remove_duplicated_spikes(sorting,censored_period_ms=0.4,method='keep_first')
sorting2=si.remove_duplicated_spikes(sorting,censored_period_ms=1.4,method='keep_first')

In [155]:
#初回はwaveformデータを保存する。
#If you want to create and save waveform, then do the following code 
# sparsity_dict = sparsity_dict=dict(method="best_channels", num_channels=8)
job_kwargs = dict(n_jobs=10, chunk_duration='1s', progress_bar=True)
we = si.extract_waveforms(rec2, sorting2, folder=folder_path / 'waveforms_kilosort4-2',
                          sparse=True, max_spikes_per_unit=500, ms_before=1.5,ms_after=2.,
                          **job_kwargs)

#もしもwaveforms_kilosort4を一度作っているのならば、こちらを実行  
# we = si.load_waveforms(folder=folder_path / 'waveforms_kilosort4')

extract waveforms shared_memory multi buffer:   0%|          | 0/5006 [00:00<?, ?it/s]

extract waveforms memmap multi buffer:   0%|          | 0/5006 [00:00<?, ?it/s]

In [156]:
#主成分分析　特徴量
pc = si.compute_principal_components(we, n_components=3, load_if_exists=True, **job_kwargs)
we.get_available_extension_names()

Fitting PCA:   0%|          | 0/71 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/71 [00:00<?, ?it/s]

['principal_components']

In [157]:
#quality metricsを計算
# _ = si.compute_spike_amplitudes(waveform_extractor=we)
_ = si.compute_correlograms(sorting2)
_ = si.compute_template_similarity(waveform_extractor=we)
_ = si.compute_quality_metrics(waveform_extractor=we, metric_names=['snr', 'isi_violation', 'isolation_distance','d_prime','l_ratio'])

Computing PCA metrics:   0%|          | 0/71 [00:00<?, ?it/s]

In [158]:
#phy形式で結果を書き込む、pc_featuresをTrueにしておくと、phy上でfeature viewが使える
si.export_to_phy(waveform_extractor=we, output_folder= folder_path/'mPhy2',
                 compute_amplitudes=False, compute_pc_features=True, copy_binary=False,
                 **job_kwargs)

# si.export_to_phy(waveform_extractor=we, output_folder= folder_path/'mPhy2')

extract PCs:   0%|          | 0/5006 [00:00<?, ?it/s]

Run:
phy template-gui  O:\seaturtle\2023\combine\gamma\20230821-2\20230821_164906.kilosort\stFolder\kilosort4\mPhy2\params.py
